In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import sys
import time
import json
from glob import glob
from PIL import Image
import pickle

from yolo_extraction import yolo8_bounding_boxes_with_distance

In [ ]:
!unzip "/content/drive/MyDrive/MSCOCO/captions2014.zip" -d "/content"
!unzip "/content/drive/MyDrive/MSCOCO/train2014.zip" -d "/content"

In [ ]:
with open("/content/annotations/captions_train2014.json", 'r') as f:
    annotations = json.load(f)

In [ ]:
# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []
all_ids = []

# create an image index
image_id_index = {}
for img in annotations['images']:
    image_id_index[img['id']] = img['file_name']

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']

    img_path = "/content/train2014/" + image_id_index[image_id]

    all_ids.append(image_id)
    all_img_name_vector.append(img_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_ids, train_captions, img_name_vector = shuffle(all_ids,
                                         all_captions,
                                         all_img_name_vector,
                                         random_state=1)

In [ ]:
train_ids = train_ids[:1000]
train_captions = train_captions[:1000]
img_name_vector = img_name_vector[:1000]

print("training captions: ", len(train_captions), ", all captions: ", len(all_captions))

training captions:  1000 , all captions:  414113


### **Preprocess images**

In [ ]:
# Create directory for inceptionV3 model if does not exist
if not os.path.exists("/content/" + "inception_model"):
    os.makedirs("/content/" + "inception_model")

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

87910968/87910968 [==============================] - 0s 0us/step


##YOLO Feature extraction

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
from tqdm import tqdm
#print(img_name_vector)
train_img_path = [ name for name in img_name_vector]
#print(train_img_path)
# Get unique images
encode_train = sorted(set(train_img_path))
#print(encode_train)

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

features_shape = 2048

for img, path in tqdm(image_dataset):
    #print(img, "####", path)
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                               (batch_features.shape[0], -1, batch_features.shape[3]))

    #for bf, p in zip(batch_features, "/content/inception_model"):
    i = 1
    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        print(path_of_feature)
        #np.save(path_of_feature, bf.numpy())
        yolo_features = yolo8_bounding_boxes_with_distance(path_of_feature)
        #print(yolo_features)
        yolo_features = np.array(yolo_features.flatten())
        yolo_features = np.pad(yolo_features, (0, features_shape - yolo_features.shape[0]), 'constant', constant_values=(0, 0)).astype(np.float32)
        combined_features = np.vstack((bf.numpy(), yolo_features)).astype(np.float32)
        np.save("/content/inception_model/inceptionModel.npy", combined_features)
        #np.save("/content/inception_model/inceptionModel.npy", bf.numpy())
        print('Pic number : ', i)
        i = i + 1


In [ ]:
from google.colab import files as FILE
FILE.download("/content/inception_model/inceptionModel.npy")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## TODO NEXT
## Clean up all caption
